### Importing a csv file to a neo4j

In [1]:
import pandas as pd
from neo4j.v1 import GraphDatabase, basic_auth

driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "neo4j"))
session = driver.session()
ethereum = pd.read_csv("./ethereum.csv", chunksize=10000, iterator=True)
for chunk in ethereum:
    for index, row in chunk.iterrows():
        session.run("MERGE (address1:Address {address:{address1}})"
                    "MERGE (address2:Address {address:{address2}})"
                    "CREATE (address1)-[to:TO]->(address2)", 
                    {"address1": row[IN_ADDRESS], "address2": row[IN_ADDRESS]})
session.close()